<a href="https://colab.research.google.com/github/meeshu/disruptions/blob/main/common_disruptions_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
!pwd
from google.colab.data_table import DataTable
DataTable.max_columns = 30

#importing data from github. switched from google drive.
url= 'https://raw.githubusercontent.com/meeshu/disruptions/refs/heads/main/data/Survey%20data%20-%20all-participants-coding-beta.csv' # Changed url to raw data url
df=pd.read_csv(url)
df.head()

/content


,pid,old-pid,gid,unique gid,goaldisrupted,old-cid,goal-statement,Goal categories,Unnamed: 8,cid,...,Disruption labor produced.2,affect-goal,change,Goal changes,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,S01,S01,goal1,S01goal1,"S01goal2, S01goal4",circ1,Financial independence,Financial Stability/Safety,NaN,circ1,...,NaN,I live far away from opportunities/ There are ...,I have shifted to trying to find work online,Changed strategy,NaN,NaN,NaN,NaN,NaN,NaN
1,S01,S01,goal1,S01goal1,NaN,circ2,Financial independence,Financial Stability/Safety,NaN,circ2,...,NaN,I feel things are hopeless,"If I were more motivated, I could be more fina...",Abandoned,NaN,NaN,NaN,NaN,NaN,NaN
2,S01,S01,goal1,S01goal1,NaN,circ3,Financial independence,Financial Stability/Safety,NaN,circ3,...,NaN,I cannot stand or walk much,"I have to work more from home, but my level of...",Changed strategy,NaN,NaN,NaN,NaN,NaN,NaN
3,S01,S01,goal2,S01goal2,NaN,circ1,Quell fears,Mental health and wellbeing,NaN,circ1,...,NaN,I am fearful of the enemy (his father),I made an appt with a dr despite wanting to ha...,Changed strategy,NaN,NaN,NaN,NaN,NaN,NaN
4,S01,S01,goal2,S01goal2,NaN,circ2,Quell fears,Mental health and wellbeing,NaN,circ2,...,NaN,Fearful I won't be able to sell home if I tried,I have to trust the universe/God,No change,NaN,NaN,NaN,NaN,NaN,NaN


# Data cleaning and prepping for analysis for manuscript

In [ ]:
# @title
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.columns = df.columns.str.rstrip()
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()
df = df.drop(columns=['old-pid'])
df = df.drop(columns=['old-cid'])
df['disruption_labor_aggregated'] = df[['disruption_labor_produced',
                                                          'disruption_labor_produced.1',
                                                          'disruption_labor_produced.2']].apply(
    lambda row: ', '.join(row.dropna().astype(str)), axis=1)

                                                          # @title
df['how-long'] = df['how-long'].apply(
    lambda x: "No disruption" if isinstance(x, str) and "no disruption" in x else x
)
df['disruption_status'] = df.apply(
    lambda x: 'No Disruption' if (x['situation_that_occurred'] == "No disruption")
    #or(x['disruption_labor_aggregated'] in ["No labor- positive disruption"])
    or (x['how-long'] in ["No disruption"])
    else 'Disruption', axis=1)

df['pid*gid*cid'] = df['pid'] + "*" + df['gid'] + "*" + df['cid']

df

,pid,gid,unique_gid,goaldisrupted,goal-statement,goal_categories,cid,uniquecid,goal_scope,rep-id,...,affect-logistical,disruption_labor_produced,disruption_labor_produced.1,disruption_labor_produced.2,affect-goal,change,goal_changes,disruption_labor_aggregated,disruption_status,pid*gid*cid
0,S01,goal1,S01goal1,"S01goal2, S01goal4",Financial independence,Financial Stability/Safety,circ1,S01goal1circ1,Long term,NaN,...,NaN,Logistical,NaN,NaN,I live far away from opportunities/ There are ...,I have shifted to trying to find work online,Changed strategy,Logistical,Disruption,S01*goal1*circ1
1,S01,goal1,S01goal1,NaN,Financial independence,Financial Stability/Safety,circ2,S01goal1circ2,Long term,NaN,...,NaN,Emotional,NaN,NaN,I feel things are hopeless,"If I were more motivated, I could be more fina...",Abandoned,Emotional,Disruption,S01*goal1*circ2
2,S01,goal1,S01goal1,NaN,Financial independence,Financial Stability/Safety,circ3,S01goal1circ3,Long term,NaN,...,NaN,Physical,NaN,NaN,I cannot stand or walk much,"I have to work more from home, but my level of...",Changed strategy,Physical,Disruption,S01*goal1*circ3
3,S01,goal2,S01goal2,NaN,Quell fears,Mental health and wellbeing,circ1,S01goal2circ1,Aspirational,r1,...,NaN,Emotional,NaN,NaN,I am fearful of the enemy (his father),I made an appt with a dr despite wanting to ha...,Changed strategy,Emotional,Disruption,S01*goal2*circ1
4,S01,goal2,S01goal2,NaN,Quell fears,Mental health and wellbeing,circ2,S01goal2circ2,Aspirational,r2,...,NaN,Emotional,NaN,NaN,Fearful I won't be able to sell home if I tried,I have to trust the universe/God,No change,Emotional,Disruption,S01*goal2*circ2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,S149,goal1,S149goal1,NaN,Save money for a newer car,Shopping,circ2,S149goal1circ2,Long term,NaN,...,"none, I had no choice. I tried looking for che...",Emotional,Financial,Logistical,"Well, instead of spending $45 on pads and $60-...",Instead of putting away $200 from my paycheck ...,Skipped,"Emotional, Financial, Logistical",Disruption,S149*goal1*circ2
758,S149,goal1,S149goal1,NaN,Save money for a newer car,Shopping,circ3,S149goal1circ3,Long term,NaN,...,na,Emotional,NaN,NaN,Instead of paying $32.99 I had to spend $55.99...,I had to be realistic and had to add more time...,Changed time,Emotional,Disruption,S149*goal1*circ3
759,S149,goal1,S149goal1,NaN,Save money for a newer car,Shopping,circ4,S149goal1circ4,Long term,NaN,...,na,Emotional,NaN,NaN,"Well, instead of paying a fair price withing m...",I had to call around and ask everyone if they ...,Changed time,Emotional,Disruption,S149*goal1*circ4
760,S149,goal2,S149goal2,NaN,workout 3-4 times a week,Physical Activity,circ1,S149goal2circ1,Medium term,NaN,...,I had to mostly work out my arms and core and ...,Physical,Emotional,NaN,it's hard to put weight on my leg when it's al...,I twisted my knee and was unable to walk strai...,Changed strategy,"Physical, Emotional",Disruption,S149*goal2*circ1


In [ ]:
# @title
disruption_data = df[df['disruption_status'] == 'Disruption']
temp_df = disruption_data

# combining categories and renaming labels to differentiate from goal categories
temp_df['situation_that_occurred'] = temp_df['situation_that_occurred'].replace({
    'Internal-Skills': 'Emotional and cognitive disruptions',
    'Internal - Feelings and thoughts': 'Emotional and cognitive disruptions',
    'Social interaction - other': 'Social interaction disruptions',
    'Social interactions - negative': 'Social interaction disruptions',
    'Pain': 'Health disruptions',
     'Diet': 'Health disruptions',
    'Sleep': 'Health disruptions',
    'Entertainment': 'Leisure disruptions',
    'Travel': 'Leisure disruptions',
    'Housework': 'Family and home responsibilities disruptions',
    'Caregiving': 'Family and home responsibilities disruptions',
    'Transportation': 'Resources disruptions',
    'Finances': 'Financial disruptions',
    'Work': 'Employment disruptions', #work or job related activities
    'Employment': 'Financial disruptions', # most similar to financial disruptions- lack of salary/ income, difficulties finding work
    'Other': 'Other disruptions',
    'Resources': 'Resources disruptions',
    'Health': 'Health disruptions',
    'Weather' : 'Weather disruptions'

})

#grouping goals
temp_df['goal_categories'] = temp_df['goal_categories'].replace({
    'Manage chronic condition': 'Health',
    'Sleep': 'Health',
    'Physical Activity': 'Health',
    'Mental health and wellbeing': 'Health',
    'Weight management': 'Health',
    'Nutrition': 'Health',
    'Health - other': 'Health',
    'Education/Learning': 'Work/Education',
    'Work/Career': 'Work/Education',
    'Financial Stability/Safety': 'Finances',
    'Shopping': 'Finances',
    'Housing': 'Finances',
    'Screentime management': 'Other',
    'Cleaning/organizing' : 'Other'
})

disruption_data = temp_df
disruption_data

<ipython-input-3-de96343f9f85>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['situation_that_occurred'] = temp_df['situation_that_occurred'].replace({
<ipython-input-3-de96343f9f85>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['goal_categories'] = temp_df['goal_categories'].replace({


,pid,gid,unique_gid,goaldisrupted,goal-statement,goal_categories,cid,uniquecid,goal_scope,rep-id,...,affect-logistical,disruption_labor_produced,disruption_labor_produced.1,disruption_labor_produced.2,affect-goal,change,goal_changes,disruption_labor_aggregated,disruption_status,pid*gid*cid
0,S01,goal1,S01goal1,"S01goal2, S01goal4",Financial independence,Finances,circ1,S01goal1circ1,Long term,NaN,...,NaN,Logistical,NaN,NaN,I live far away from opportunities/ There are ...,I have shifted to trying to find work online,Changed strategy,Logistical,Disruption,S01*goal1*circ1
1,S01,goal1,S01goal1,NaN,Financial independence,Finances,circ2,S01goal1circ2,Long term,NaN,...,NaN,Emotional,NaN,NaN,I feel things are hopeless,"If I were more motivated, I could be more fina...",Abandoned,Emotional,Disruption,S01*goal1*circ2
2,S01,goal1,S01goal1,NaN,Financial independence,Finances,circ3,S01goal1circ3,Long term,NaN,...,NaN,Physical,NaN,NaN,I cannot stand or walk much,"I have to work more from home, but my level of...",Changed strategy,Physical,Disruption,S01*goal1*circ3
3,S01,goal2,S01goal2,NaN,Quell fears,Health,circ1,S01goal2circ1,Aspirational,r1,...,NaN,Emotional,NaN,NaN,I am fearful of the enemy (his father),I made an appt with a dr despite wanting to ha...,Changed strategy,Emotional,Disruption,S01*goal2*circ1
4,S01,goal2,S01goal2,NaN,Quell fears,Health,circ2,S01goal2circ2,Aspirational,r2,...,NaN,Emotional,NaN,NaN,Fearful I won't be able to sell home if I tried,I have to trust the universe/God,No change,Emotional,Disruption,S01*goal2*circ2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,S149,goal1,S149goal1,NaN,Save money for a newer car,Finances,circ2,S149goal1circ2,Long term,NaN,...,"none, I had no choice. I tried looking for che...",Emotional,Financial,Logistical,"Well, instead of spending $45 on pads and $60-...",Instead of putting away $200 from my paycheck ...,Skipped,"Emotional, Financial, Logistical",Disruption,S149*goal1*circ2
758,S149,goal1,S149goal1,NaN,Save money for a newer car,Finances,circ3,S149goal1circ3,Long term,NaN,...,na,Emotional,NaN,NaN,Instead of paying $32.99 I had to spend $55.99...,I had to be realistic and had to add more time...,Changed time,Emotional,Disruption,S149*goal1*circ3
759,S149,goal1,S149goal1,NaN,Save money for a newer car,Finances,circ4,S149goal1circ4,Long term,NaN,...,na,Emotional,NaN,NaN,"Well, instead of paying a fair price withing m...",I had to call around and ask everyone if they ...,Changed time,Emotional,Disruption,S149*goal1*circ4
760,S149,goal2,S149goal2,NaN,workout 3-4 times a week,Health,circ1,S149goal2circ1,Medium term,NaN,...,I had to mostly work out my arms and core and ...,Physical,Emotional,NaN,it's hard to put weight on my leg when it's al...,I twisted my knee and was unable to walk strai...,Changed strategy,"Physical, Emotional",Disruption,S149*goal2*circ1


# disruptions to more than 1 participant goals

In [9]:
# prompt: filter disruption_data where repid is not NaN

disruption_data_filtered = disruption_data[disruption_data['rep-id'].notna()]
disruption_data_filtered [['pid','gid','rep-id','goal-statement','goal_categories','uniquecid','circumstance','situation_that_occurred','how-long']]


,pid,gid,rep-id,goal-statement,goal_categories,uniquecid,circumstance,situation_that_occurred,how-long
3,S01,goal2,r1,Quell fears,Health,S01goal2circ1,Son is injured,Family and home responsibilities disruptions,multiple days to a week
4,S01,goal2,r2,Quell fears,Health,S01goal2circ2,Lack of financial independence,Financial disruptions,longer than a year
6,S01,goal2,r3,Quell fears,Health,S01goal2circ4,Storm coming,Weather disruptions,multiple days to a week
10,S01,goal4,r1,Make my children proud of me,Social connection,S01goal4circ1,I feel hopeless in battling my son's injury,Emotional and cognitive disruptions,multiple days to a week
13,S01,goal4,r3,Make my children proud of me,Social connection,S01goal4circ4,I am worried about the storm,Weather disruptions,multiple days to a week
...,...,...,...,...,...,...,...,...,...
703,S135,goal2,r36,save for satellite internet,Finances,S135goal2circ1,unplanned dental emergency,Health disruptions,multiple months to a year
727,S142,goal1,r37,Read my bible daily,Health,S142goal1circ1,Waking up late,Health disruptions,an hour or less
729,S142,goal2,r37,Time management,Other,S142goal2circ1,Waking up late,Health disruptions,longer than a year
742,S146,goal1,r38,Financial wellness,Finances,S146goal1circ1,Habitual overspending,Financial disruptions,longer than a year


## number of participants who reported a disruption to more than 1 goal

In [10]:
# prompt: count of unique pid in disruption_data_filtered

unique_pid_count = disruption_data_filtered['pid'].nunique()
print(f"The number of unique PIDs in disruption_data_filtered is: {unique_pid_count}")

The number of unique PIDs in disruption_data_filtered is: 32


filtering to unique disruptions and identifying what categories they are in

In [13]:
# prompt: filter to unique rep-id in disruption_data_filtered

# Use drop_duplicates to keep only the first occurrence of each unique 'rep-id'
repeating_disruptions = disruption_data_filtered.drop_duplicates(subset=['rep-id'])
repeating_disruptions

,pid,gid,unique_gid,goaldisrupted,goal-statement,goal_categories,cid,uniquecid,goal_scope,rep-id,...,affect-logistical,disruption_labor_produced,disruption_labor_produced.1,disruption_labor_produced.2,affect-goal,change,goal_changes,disruption_labor_aggregated,disruption_status,pid*gid*cid
3,S01,goal2,S01goal2,NaN,Quell fears,Health,circ1,S01goal2circ1,Aspirational,r1,...,NaN,Emotional,NaN,NaN,I am fearful of the enemy (his father),I made an appt with a dr despite wanting to ha...,Changed strategy,Emotional,Disruption,S01*goal2*circ1
4,S01,goal2,S01goal2,NaN,Quell fears,Health,circ2,S01goal2circ2,Aspirational,r2,...,NaN,Emotional,NaN,NaN,Fearful I won't be able to sell home if I tried,I have to trust the universe/God,No change,Emotional,Disruption,S01*goal2*circ2
6,S01,goal2,S01goal2,NaN,Quell fears,Health,circ4,S01goal2circ4,Aspirational,r3,...,NaN,Emotional,NaN,NaN,It's just one more worry I don't need,I changed plans and tried to prep the house on...,Changed strategy,Emotional,Disruption,S01*goal2*circ4
19,S02,goal1,S02goal1,NaN,strengthening friendships,Social connection,circ3,S02goal1circ3,Aspirational,r4,...,NaN,Emotional,NaN,NaN,NaN,I had to pause social outings,Paused,Emotional,Disruption,S02*goal1*circ3
29,S03,goal1,S03goal1,NaN,Get my own place to live,Finances,circ1,S03goal1circ1,Long term,r5,...,NaN,Emotional,NaN,NaN,Makes me feel like I can’t count on anyone,I haven’t I’m just working on getting the mone...,Changed strategy,Emotional,Disruption,S03*goal1*circ1
31,S04,goal1,S04goal1,NaN,Excercising,Health,circ1,S04goal1circ1,Medium term,r6,...,NaN,Emotional,NaN,NaN,stress from work,To get more time to commute to work,Skipped,Emotional,Disruption,S04*goal1*circ1
41,S05,goal2,S05goal2,NaN,Set aside $400 a month to spend on social events,Recreation,circ1,S05goal2circ1,Medium term,r7,...,NaN,Emotional,NaN,NaN,NaN,It just didn't get met and i had to stay at ho...,Abandoned,Emotional,Disruption,S05*goal2*circ1
45,S06,goal1,S06goal1,NaN,advancing my knowledge at my work place,Work/Education,circ1,S06goal1circ1,Long term,r8,...,NaN,Emotional,NaN,NaN,it was beneath my cost of living,i stayed focus on the long term goal.,No change,Emotional,Disruption,S06*goal1*circ1
49,S06,goal1,S06goal1,NaN,advancing my knowledge at my work place,Work/Education,circ5,S06goal1circ5,Long term,r9,...,NaN,Emotional,NaN,NaN,my ego is a distraction,i used my ego in a way that i could thrive off...,Changed strategy,Emotional,Disruption,S06*goal1*circ5
82,S09,goal1,S09goal1,NaN,Save for bigger house,Finances,circ3,S09goal1circ3,Long term,r10,...,NaN,Financial,NaN,NaN,Now we have less to save,Less for saving and more for bills for the tim...,Changed strategy,Financial,Disruption,S09*goal1*circ3


In [17]:
# temporary code cleaning--- REMOVE THIS WHEN RUNNING ON GAMMA dataset
# change r1 rep-id situation_that_occurred value to "Social interaction disruptions"
repeating_disruptions.loc[repeating_disruptions['rep-id'] == 'r1', 'situation_that_occurred'] = 'Social interaction disruptions'

In [18]:
# prompt: group repeating_disruptions by situation_that_occurred

grouped_disruptions = repeating_disruptions.groupby('situation_that_occurred')['rep-id'].count()
grouped_disruptions

,rep-id
situation_that_occurred,
Emotional and cognitive disruptions,4
Employment disruptions,7
Family and home responsibilities disruptions,2
Financial disruptions,9
Health disruptions,5
Leisure disruptions,4
Resources disruptions,1
Social interaction disruptions,5
Weather disruptions,1


In [19]:
# prompt: count of rows for disruption_data group by situation_that_occurred

# Calculate the count of rows for disruption_data grouped by situation_that_occurred
row_counts = disruption_data.groupby('situation_that_occurred')['pid*gid*cid'].count()

row_counts

,pid*gid*cid
situation_that_occurred,
Emotional and cognitive disruptions,147
Employment disruptions,75
Family and home responsibilities disruptions,22
Financial disruptions,137
Health disruptions,104
Leisure disruptions,35
Other disruptions,16
Resources disruptions,68
Social interaction disruptions,97


In [20]:
# prompt: merge row_counts and grouped_disruptions and add new column with grouped_disruptions as a % of row_counts

# Merge row_counts and grouped_disruptions based on the index (situation_that_occurred)
merged_df = pd.merge(row_counts, grouped_disruptions, left_index=True, right_index=True, how='left')

# Rename columns for clarity
merged_df = merged_df.rename(columns={'pid*gid*cid': 'Total_Disruptions', 'rep-id': 'Unique_Disruption_MultipleGoals'})

# Calculate the percentage of unique disruptions out of total disruptions
merged_df['Percentage_Unique'] = (merged_df['Unique_Disruption_MultipleGoals'] / merged_df['Total_Disruptions']) * 100

# Replace NaN values in Percentage_Unique with 0
merged_df['Percentage_Unique'] = merged_df['Percentage_Unique'].fillna(0)

merged_df

,Total_Disruptions,Unique_Disruption_MultipleGoals,Percentage_Unique
situation_that_occurred,,,
Emotional and cognitive disruptions,147,4.0,2.721088
Employment disruptions,75,7.0,9.333333
Family and home responsibilities disruptions,22,2.0,9.090909
Financial disruptions,137,9.0,6.569343
Health disruptions,104,5.0,4.807692
Leisure disruptions,35,4.0,11.428571
Other disruptions,16,NaN,0.000000
Resources disruptions,68,1.0,1.470588
Social interaction disruptions,97,5.0,5.154639
